In [22]:
import pandas as pd

df=pd.read_csv("./data.csv")
df

,Site_ID,Latitude_Degrees,Longitude_Degrees,Ocean_Name,Ecoregion_Name,Exposure,Turbidity,Cyclone_Frequency,Date_Month,Date_Year,...,Temperature_Mean,Windspeed,SSTA,SSTA_DHW,TSA,TSA_DHWMean,Date,Site_Comments,Sample_Comments,Bleaching_condition
0,29,-23.4439,151.9286,Pacific,Southern Great Barrier Reef,Sheltered,0.0731,51.57,2,1998,...,297.35,5,5.48,31.79,5.24,10.05,15-02-1998,Acropora colonies (staghorn corals) are mainly...,Acropora colonies (staghorn corals) are mainly...,Bleaching percent averaged from code: Severe (...
1,63,-18.7502,147.2688,Pacific,Central and northern Great Barrier Reef,Sheltered,0.0458,44.20,2,1998,...,299.18,2,2.05,4.43,2,0.44,15-02-1998,80% of corals bleached,80% of corals bleached,Bleaching percent averaged from code: Severe (...
2,65,-17.2661,146.3636,Pacific,Central and northern Great Barrier Reef,Sheltered,0.0755,46.35,2,1998,...,299.52,3,0.85,2.49,0.73,0.4,15-02-1998,significant bleaching,significant bleaching,Bleaching percent averaged from code: Severe (...
3,81,-20.1930,150.0820,Pacific,Central and northern Great Barrier Reef,Sheltered,0.0743,48.00,3,2002,...,298.52,8,0.46,3.14,-0.02,0.24,15-03-2002,lots of white on slope,lots of white on slope,Bleaching percent averaged from code: Severe (...
4,85,-20.1559,149.0719,Pacific,Central and northern Great Barrier Reef,Sheltered,0.0627,46.13,3,2002,...,298.55,7,0.41,4.91,-0.09,0.37,15-03-2002,ie Minstrel rock on nth end. South Deloraine w...,ie Minstrel rock on nth end. South Deloraine w...,Bleaching percent averaged from code: Severe (...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1247,9243,24.6200,-82.8600,Atlantic,Bahamas and Florida Keys,Exposed,0.0581,71.57,10,2010,...,299.34,6,-0.49,2.54,-2.81,0.12,15-10-2010,Average Hard coral: 5% Soft coral: 28% Bleache...,Average Hard coral: 5% Soft coral: 28% Bleache...,Bleaching percentage averaged from code: Mild ...
1248,9338,18.3200,-64.7330,Atlantic,Hispaniola Puerto Rico and Lesser Antilles,Sheltered,0.0431,65.66,11,2003,...,300.33,3,-0.49,2.38,-1.16,0.3,15-11-2003,Some reports of bleaching this past week aroun...,Some reports of bleaching this past week aroun...,Bleaching percentage averaged from code: Mild ...
1249,9564,10.7810,-68.3110,Atlantic,Netherlands Antilles and south Caribbean,Sheltered,0.5181,54.00,11,1987,...,300.41,4,-2.27,2.33,-2.7,0.45,15-11-1987,13.5% of coral cover was pale; 4.1% bleached (...,13.5% of coral cover was pale; 4.1% bleached (...,Bleaching percentage averaged from code: Mild ...
1250,9566,10.8500,-68.2100,Atlantic,Netherlands Antilles and south Caribbean,Exposed,0.0695,54.05,12,1987,...,300.13,4,0.5,5.1,-0.98,0.44,15-12-1987,Millepora alcicornis - 60% of colonies bleach...,Millepora alcicornis - 60% of colonies bleach...,Bleaching percentage averaged from code: Mild ...


In [23]:
def encode_bleaching_severity(text):
    if isinstance(text, str):
        text = text.lower()
        if 'severe' in text:
            return 2
        elif 'mild' in text:
            return 1
        elif 'low' in text:
            return 0
    return None
df['Bleaching_Severity_Int'] = df['Bleaching_condition'].apply(encode_bleaching_severity)
cols = ['Temperature_Mean', 'SSTA', 'SSTA_DHW', 'Windspeed']
for col in cols:
    df[col] = df[col].astype(str).str.strip().str.lower()
import numpy as np

bad_values = ['nd', 'nan', 'none', 'null', '']

df[cols] = df[cols].replace(bad_values, np.nan)
for col in cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')


In [24]:
from sklearn.preprocessing import MinMaxScaler

features = ['Temperature_Mean', 'SSTA', 'SSTA_DHW', 'Windspeed', 'Bleaching_Severity_Int']

scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df[features])

df_scaled = pd.DataFrame(df_scaled, columns=features)

In [25]:
df['Thermal_Stress_Index'] = (
    0.4 * df_scaled['Temperature_Mean'] +
    0.3 * df_scaled['SSTA'] +
    0.3 * df_scaled['SSTA_DHW']
)

In [26]:
df['Bleaching_Impact'] = (
    0.6 * df_scaled['Bleaching_Severity_Int'] +
    0.4 * df['Thermal_Stress_Index']
)

In [27]:
df['Wind_Mitigation'] = 1 - df_scaled['Windspeed']

df['Bleaching_Duration_weeks'] = (
    df['Bleaching_Impact'] * df['Wind_Mitigation'] * 16
)

df['Bleaching_Duration_weeks'] = (
    df['Bleaching_Duration_weeks']
    .fillna(0)        # 🔑 FIX
    .clip(0, 16)
    .round()
    .astype(int)
)

In [28]:
df

,Site_ID,Latitude_Degrees,Longitude_Degrees,Ocean_Name,Ecoregion_Name,Exposure,Turbidity,Cyclone_Frequency,Date_Month,Date_Year,...,TSA_DHWMean,Date,Site_Comments,Sample_Comments,Bleaching_condition,Bleaching_Severity_Int,Thermal_Stress_Index,Bleaching_Impact,Wind_Mitigation,Bleaching_Duration_weeks
0,29,-23.4439,151.9286,Pacific,Southern Great Barrier Reef,Sheltered,0.0731,51.57,2,1998,...,10.05,15-02-1998,Acropora colonies (staghorn corals) are mainly...,Acropora colonies (staghorn corals) are mainly...,Bleaching percent averaged from code: Severe (...,2.0,0.691134,0.876454,0.6,8
1,63,-18.7502,147.2688,Pacific,Central and northern Great Barrier Reef,Sheltered,0.0458,44.20,2,1998,...,0.44,15-02-1998,80% of corals bleached,80% of corals bleached,Bleaching percent averaged from code: Severe (...,2.0,0.477502,0.791001,0.9,11
2,65,-17.2661,146.3636,Pacific,Central and northern Great Barrier Reef,Sheltered,0.0755,46.35,2,1998,...,0.4,15-02-1998,significant bleaching,significant bleaching,Bleaching percent averaged from code: Severe (...,2.0,0.436266,0.774506,0.8,10
3,81,-20.1930,150.0820,Pacific,Central and northern Great Barrier Reef,Sheltered,0.0743,48.00,3,2002,...,0.24,15-03-2002,lots of white on slope,lots of white on slope,Bleaching percent averaged from code: Severe (...,2.0,0.393670,0.757468,0.3,4
4,85,-20.1559,149.0719,Pacific,Central and northern Great Barrier Reef,Sheltered,0.0627,46.13,3,2002,...,0.37,15-03-2002,ie Minstrel rock on nth end. South Deloraine w...,ie Minstrel rock on nth end. South Deloraine w...,Bleaching percent averaged from code: Severe (...,2.0,0.402988,0.761195,0.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1247,9243,24.6200,-82.8600,Atlantic,Bahamas and Florida Keys,Exposed,0.0581,71.57,10,2010,...,0.12,15-10-2010,Average Hard coral: 5% Soft coral: 28% Bleache...,Average Hard coral: 5% Soft coral: 28% Bleache...,Bleaching percentage averaged from code: Mild ...,1.0,0.384432,0.153773,0.5,1
1248,9338,18.3200,-64.7330,Atlantic,Hispaniola Puerto Rico and Lesser Antilles,Sheltered,0.0431,65.66,11,2003,...,0.3,15-11-2003,Some reports of bleaching this past week aroun...,Some reports of bleaching this past week aroun...,Bleaching percentage averaged from code: Mild ...,1.0,0.416035,0.166414,0.8,2
1249,9564,10.7810,-68.3110,Atlantic,Netherlands Antilles and south Caribbean,Sheltered,0.5181,54.00,11,1987,...,0.45,15-11-1987,13.5% of coral cover was pale; 4.1% bleached (...,13.5% of coral cover was pale; 4.1% bleached (...,Bleaching percentage averaged from code: Mild ...,1.0,0.356999,0.142800,0.7,2
1250,9566,10.8500,-68.2100,Atlantic,Netherlands Antilles and south Caribbean,Exposed,0.0695,54.05,12,1987,...,0.44,15-12-1987,Millepora alcicornis - 60% of colonies bleach...,Millepora alcicornis - 60% of colonies bleach...,Bleaching percentage averaged from code: Mild ...,1.0,0.459059,0.183624,0.7,2


In [34]:
df_clean = df.drop(
    columns=[
        'Sample_Comments',
        'Bleaching_condition',
        'Site_Comments'
    ]
)
df_clean.to_csv('coral_bleaching_cleaned.csv', index=False)